In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [2]:
data=pd.read_csv("HotelsFinalDataset.csv")

In [4]:
data.tail()

,state,city,HotelName,Address,PhoneNumber,HotelWebsiteUrl,latitude,longitude,HotelRating,Spa,...,Wheel-chair accessible,Snack Bar,Complementary breakfast,Fitness Center,Eco-friendly,24 hr front desk,Valet Parking,Open bar,Child Friendly,smoking
3581,TN,woodbury,Longwood Country Inn,1204 Main St S WoodburyConnecticut 06798,2032660800,https://www.booking.com/hotel/us/longwood-coun...,0.970634,1.075692,3.0,0,...,0,0,1,0,0,0,1,0,0,0
3582,TN,woodbury,Country Loft Bed and Breakfast,#5 Old Sherman Hill Road WoodburyConnecticut 0...,2032664500,https://www.booking.com/hotel/us/country-loft-...,0.974358,1.075776,3.0,0,...,0,0,1,0,0,0,1,0,0,0
3583,NJ,woodbury,Fairfield Inn by Marriott Deptford,1160 Hurffville Rd DeptfordNew Jersey 08096,8566869050,http://www.marriott.de/hotels/travel/phldf-fai...,0.687956,0.969901,2.0,0,...,1,0,1,1,0,1,1,1,0,0
3584,NJ,woodbury,Courtyard Deptford,1251 Hurffville Road Deptford TownshipNew Jers...,8562321500,http://www.marriott.com/hotels/travel/phlpt-co...,0.686796,0.969905,3.0,0,...,1,0,1,1,0,1,1,0,0,0
3585,NJ,woodbury,"Hampton Inn & Suites Deptford, NJ",1253 Hurffville Road,8004458667,https://www.booking.com/hotel/us/hampton-inn--...,0.686878,0.969860,2.0,0,...,1,0,1,1,0,1,1,0,0,0


In [6]:
# unique_states = data['state'].unique()
# print("Unique States:", unique_states)

Unique States: ['TN' 'IL' 'CA' 'PA' 'NJ' 'ID' 'NC' 'MO' 'CO' 'FL' 'IN' 'AZ' 'LA' 'DE'
 'NV']


In [8]:
# unique_cities = data['city'].unique()
# print("Unique Cities:", unique_cities)

Unique Cities: ['antioch' 'audubon' 'barrington' 'bellmawr' 'berlin' 'blackwood' 'boise'
 'boone' 'bordentown' 'bridgeton' 'brooklawn' 'burlington' 'camden'
 'carpinteria' 'cherry hill' 'chester' 'cinnaminson' 'clayton'
 'clearwater' 'collegeville' 'columbus' 'concordville' 'davidson'
 'doylestown' 'dunedin' 'eagle' 'fairview heights' 'fayette' 'florence'
 'franklin' 'gibbstown' 'glassboro' 'glendora' 'gloucester city' 'goleta'
 'hammonton' 'hancock' 'hernando' 'hillsborough' 'indianapolis'
 'jefferson' 'kenner' 'kulpsville' 'lambertville' 'lawrenceville'
 'lumberton' 'madison' 'malvern' 'mansfield' 'maple shade' 'marion'
 'mendenhall' 'meridian' 'monroe' 'monroeville' 'mount ephraim'
 'mount holly' 'mount laurel' 'nashville' 'nevada' 'new castle'
 'new orleans' 'newark' 'norristown' 'orange' 'palm harbor' 'palmyra'
 'pemberton' 'pennsville' 'philadelphia' 'reno' 'riverside' 'runnemede'
 'rutherford' 'salem' 'santa barbara' 'sparks' 'st. charles' 'st. louis'
 'st. petersburg' 'swedesbo

In [ ]:
data.columns

Index(['state', 'city', 'HotelName', 'Address', 'PhoneNumber',
       'HotelWebsiteUrl', 'latitude', 'longitude', 'HotelRating', 'Spa',
       'Airport shuttle', 'Private Lounge', 'Business Center',
       'Contactless check', 'Pool', 'Pet friendly', 'Free Wifi',
       'Wheel-chair accessible', 'Snack Bar', 'Complementary breakfast',
       'Fitness Center', 'Eco-friendly', '24 hr front desk', 'Valet Parking',
       'Open bar', 'Child Friendly', 'smoking'],
      dtype='object')

In [23]:
# Filter rows where the city is 'Auduon'
filtered_rows = data[data['city'] == 'barrington']

print(filtered_rows)


Empty DataFrame
Columns: [state, city, HotelName, Address, PhoneNumber, HotelWebsiteUrl, latitude, longitude, HotelRating, Spa, Airport shuttle, Private Lounge, Business Center, Contactless check, Pool, Pet friendly, Free Wifi, Wheel-chair accessible, Snack Bar, Complementary breakfast, Fitness Center, Eco-friendly, 24 hr front desk, Valet Parking, Open bar, Child Friendly, smoking, rating]
Index: []

[0 rows x 28 columns]


# Step 1: Preprocessing

In [9]:
label_state=LabelEncoder()
label_city=LabelEncoder()
data['state']=label_state.fit_transform(data['state'])
data['city']=label_city.fit_transform(data['city'])

In [10]:
data.head(2)

,state,city,HotelName,Address,PhoneNumber,HotelWebsiteUrl,latitude,longitude,HotelRating,Spa,...,Wheel-chair accessible,Snack Bar,Complementary breakfast,Fitness Center,Eco-friendly,24 hr front desk,Valet Parking,Open bar,Child Friendly,smoking
0,14,0,Hampton Inn Nashville I 24 Hickory Hollow,210 Crossings Place AntiochTennessee 37013,6157319911,http://hamptoninn.hilton.com/en/hp/hotels/inde...,0.048447,0.320218,2.0,0,...,1,0,1,1,0,1,1,0,0,0
1,14,0,Days Inn Nashville Southeast,510B Collins Park Drive Antioch,6157319911,https://www.booking.com/hotel/us/days-inn-nash...,0.048452,0.319803,3.0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
facilities=['Spa','Airport shuttle', 'Private Lounge', 'Business Center',
       'Contactless check', 'Pool', 'Pet friendly', 'Free Wifi',
       'Wheel-chair accessible', 'Complementary breakfast',
       'Fitness Center', '24 hr front desk', 'Valet Parking',
       'Open bar', 'Child Friendly', 'smoking']

In [12]:
data[facilities] = data[facilities].apply(pd.to_numeric, errors='coerce', downcast='integer')


In [13]:
data['rating'] = data['HotelRating']
hotel_features = data[['state', 'city'] + facilities + ['rating']]

# Normalize the feature vector (optional)
hotel_features_norm = (hotel_features - hotel_features.mean()) / hotel_features.std()

In [14]:
cos_sim = cosine_similarity(hotel_features_norm)


In [15]:
# Step 4: Collaborative Filtering (Matrix Factorization)
interaction_matrix = pd.pivot_table(data, index='HotelName', columns='state', values='HotelRating', aggfunc='count', fill_value=0)
interaction_matrix_sparse = csr_matrix(interaction_matrix)


In [16]:
svd = TruncatedSVD(n_components=5)
svd_matrix = svd.fit_transform(interaction_matrix_sparse)


In [17]:
item_item_similarity = cosine_similarity(svd_matrix)

In [18]:


# Step 5: Hybrid Recommendation System (for Real-Time Input)
def hybrid_recommendation(user_state, user_city, user_facilities, alpha=0.7):
    # Step 5.1: Filter hotels based on user input (state, city, facilities)
    filtered_hotels = data[
        (data['state'] == label_state.transform([user_state])[0]) &
        (data['city'] == label_city.transform([user_city])[0])
    ]

    # Check facilities
    for facility in user_facilities:
        filtered_hotels = filtered_hotels[filtered_hotels[facility] == 1]

    # Step 5.2: Content-based filtering (Cosine similarity score)
    filtered_hotel_indices = filtered_hotels.index
    content_scores = cos_sim[filtered_hotel_indices].mean(axis=1)

    # Step 5.3: Collaborative filtering (Item-item similarity score)
    collaborative_scores = item_item_similarity[filtered_hotel_indices].mean(axis=1)

    # Step 5.4: Combine the scores using weighted hybrid method
    final_scores = alpha * content_scores + (1 - alpha) * collaborative_scores

    # Sort by final score (descending order)
    sorted_indices = np.argsort(final_scores)[::-1]

    # Step 5.5: Return the top N recommendations (top 5 for example)
    top_recommendations = filtered_hotels.iloc[sorted_indices[:5]]

    return top_recommendations[['HotelName', 'state', 'city', 'HotelRating']]

# Example Usage (Real-Time Input)
user_state = 'TN'  # Input from user
user_city = 'antioch'  # Input from user
user_facilities = ['Spa', 'Fitness Center']  # Input from user (facilities they want)

recommendations = hybrid_recommendation(user_state, user_city, user_facilities, alpha=0.7)
print(recommendations)


              HotelName  state  city  HotelRating
2  Comfort Inn & Suites     14     0          2.0


In [19]:
import numpy as np

# Step 5: Hybrid Recommendation System (for Real-Time Input)
def hybrid_recommendation(user_state, user_city, user_facilities, alpha=0.7, top_n=3):
    # Step 5.1: Filter hotels based on user input (state, city, facilities)

    # Ensure the states and cities are transformed correctly (handle unseen states/cities)
    try:
        transformed_state = label_state.transform([user_state])[0]
        transformed_city = label_city.transform([user_city])[0]
    except ValueError:
        raise ValueError(f"Unrecognized state '{user_state}' or city '{user_city}'.")

    # Filter by state and city
    filtered_hotels = data[
        (data['state'] == transformed_state) &
        (data['city'] == transformed_city)
    ]

    # Check facilities - Relax the condition to allow hotels with at least one matching facility
    if user_facilities:
        filtered_hotels = filtered_hotels[filtered_hotels[user_facilities].sum(axis=1) > 0]

    # If no hotels match the filters, provide a fallback
    if filtered_hotels.empty:
        print("No hotels match your criteria. Expanding the search with less restrictive filters.")
        # Fallback: Relax facility filtering or use only state/city criteria
        filtered_hotels = data[
            (data['state'] == transformed_state) &
            (data['city'] == transformed_city)
        ]
        if filtered_hotels.empty:
            # If no hotels match state and city, try the entire state
            filtered_hotels = data[data['state'] == transformed_state]
            print("No hotels found in the city, expanding search to the entire state.")

    # Step 5.2: Content-based filtering (Cosine similarity score)
    # Get the indices of the filtered hotels
    filtered_hotel_indices = filtered_hotels.index

    # Calculate content-based scores (average similarity across all filtered hotels)
    content_scores = cos_sim[filtered_hotel_indices].mean(axis=1)

    # Step 5.3: Collaborative filtering (Item-item similarity score)
    collaborative_scores = item_item_similarity[filtered_hotel_indices].mean(axis=1)

    # Step 5.4: Combine the scores using a weighted hybrid method
    final_scores = alpha * content_scores + (1 - alpha) * collaborative_scores

    # Sort by final score (highest score first)
    sorted_indices = np.argsort(final_scores)[::-1]

    # Step 5.5: Ensure at least 3 recommendations (top 3 for example)
    top_n_recommendations = sorted_indices[:max(top_n, 3)]  # Ensure at least 3 hotels

    # If fewer than 3 recommendations exist, return all available hotels
    if len(top_n_recommendations) < 3:
        print("Warning: Less than 3 recommendations found. Returning all available.")
        top_n_recommendations = sorted_indices[:len(filtered_hotels)]  # Return all available

    # Return the top N recommendations (at least 3)
    top_recommendations = filtered_hotels.iloc[top_n_recommendations]

    return top_recommendations[['HotelName', 'state', 'city', 'HotelRating']]




In [24]:
# Example Usage (Real-Time Input)
user_state = 'MO'  # Input from user
user_city = 'bridgeton'  # Input from user
user_facilities = ['Spa', 'Fitness Center']  # Input from user (facilities they want)

recommendations = hybrid_recommendation(user_state, user_city, user_facilities, alpha=0.7, top_n=3)
print(recommendations)

                             HotelName  state  city  HotelRating
224  Mainstay Suites St. Louis Airport      9     9          2.0
211   Embassy Suites St. Louis Airport      9     9          3.0
225         Sleep Inn St Louis Airport      9     9          2.0
